# About.
- Summary of `logging` library.
- https://docs.python.org/3/library/logging.html
- History:
  - 2025.8.12, 1st summary.

# 1. Summary.

## 1.1. Basic Structure.
- `logger` : ***what*** to talk.
- `handler` : ***where*** to talk.
- `formatter`: ***how*** to talk.

## 1.2. Level.
- `debug`.
- `info`.
- `warning`.
- `error`.
- `critical`.

## 1.3. Types of Handler.

| Handler class | Purpose / destination | Module |
|---|---|---|
| `StreamHandler` | Writes records to a stream (e.g., `sys.stdout`, `sys.stderr`). | `logging` |
| `FileHandler` | Writes records to a disk file. | `logging` |
| `NullHandler` | Swallows records (no output); useful for libraries. | `logging` |
| `WatchedFileHandler` | Like `FileHandler` but reopens file if log rotated externally (Unix). | `logging.handlers` |
| `BaseRotatingHandler` | Base class for rotating file handlers; not used directly. | `logging.handlers` |
| `RotatingFileHandler` | Rotates files by size with backups. | `logging.handlers` |
| `TimedRotatingFileHandler` | Rotates files at fixed time intervals. | `logging.handlers` |
| `SocketHandler` | Sends records to a TCP socket. | `logging.handlers` |
| `DatagramHandler` | Sends records over UDP. | `logging.handlers` |
| `SysLogHandler` | Sends records to local/remote syslog. | `logging.handlers` |
| `NTEventLogHandler` | Sends records to the Windows Event Log. | `logging.handlers` |
| `SMTPHandler` | Emails records via SMTP. | `logging.handlers` |
| `BufferingHandler` | Abstract; buffers records in memory. | `logging.handlers` |
| `MemoryHandler` | Buffers then flushes to a target handler. | `logging.handlers` |
| `HTTPHandler` | Sends records to an HTTP server (GET/POST). | `logging.handlers` |
| `QueueHandler` | Enqueues records to `queue`/`multiprocessing`. | `logging.handlers` |


## 1.4. Arguments in Formatter.

| Placeholder | Meaning | Type | Notes |
|---|---|---|---|
| `%(asctime)s` | Human-readable time the record was created. | str | Controlled by `datefmt`. |
| `%(created)f` | Epoch time when record was created. | float | Seconds since epoch. |
| `%(filename)s` | Filename portion of `pathname`. | str | e.g., `app.py`. |
| `%(funcName)s` | Function or method name. | str |  |
| `%(levelname)s` | Text level name. | str | e.g., `INFO`. |
| `%(levelno)s` | Numeric level. | int | e.g., `20`. |
| `%(lineno)d` | Source line number. | int |  |
| `%(message)s` | Rendered log message (`msg % args`). | str | Set by `Formatter.format()`. |
| `%(module)s` | Module name (from filename). | str |  |
| `%(msecs)d` | Millisecond part of `created`. | int |  |
| `%(name)s` | Logger name. | str | Commonly `__name__`. |
| `%(pathname)s` | Full path of source file. | str |  |
| `%(process)d` | Process ID. | int |  |
| `%(processName)s` | Process name. | str ||
| `%(relativeCreated)d` | Milliseconds since logging module loaded. | int |  |
| `%(thread)d` | Thread ID. | int |  |
| `%(threadName)s` | Thread name. | str |  |
| `%(taskName)s` | `asyncio.Task` name. | str ||


# 2. Usage.

## 2.1. Simple Logging.

In [ ]:
import logging

# 1. Logger.
logger = logging.getLogger(name = 'logger_debug')
logger.setLevel(logging.DEBUG)

# 2. Handler.
if logger.hasHandlers():
    logger.handlers.clear()
    
handler = logging.StreamHandler()
handler.setLevel(logging.INFO)

# 3. Formatter.
formatter = logging.Formatter('[%(asctime)s] [%(levelname)s] %(message)s', 
                               datefmt='%Y-%m-%d %H:%M:%S')

# Add.
handler.setFormatter(formatter)
logger.addHandler(handler)

# Try.
logger.debug('D')    # Does not print cuz handler logging level is INFO.
logger.info('a')

[2025-08-12 19:10:52] [INFO] a


## 2.2. Connected with `Slack`.

In [ ]:
# Custom Slack logging handler using Incoming Webhooks.
# - No extra dependencies; uses urllib.
# - Supports dry_run for local testing.
# - Includes optional de-duplication filter to suppress repeats within a time window.

import logging
import json
import time
import os
from typing import Optional, Dict, Any
from urllib.request import Request, urlopen
from urllib.error import URLError, HTTPError


class DedupFilter(logging.Filter):
    """Filter that suppresses identical messages within a time window."""
    def __init__(self, window_seconds: int = 60, key: str = "message"):
        super().__init__()
        self.window_seconds = window_seconds
        self.key = key
        self._last_sent: Dict[str, float] = {}

    def filter(self, record: logging.LogRecord) -> bool:
        now = time.time()
        if self.key == "message":
            k = record.getMessage()
        elif self.key == "name+message":
            k = f"{record.name}:{record.getMessage()}"
        else:
            k = f"{record.levelno}:{record.name}:{record.getMessage()}"
        last = self._last_sent.get(k, 0.0)
        if now - last < self.window_seconds:
            return False
        self._last_sent[k] = now
        return True


class SlackWebhookHandler(logging.Handler):
    """Logging handler that posts formatted records to Slack via Incoming Webhook."""
    def __init__(
        self,
        webhook_url: str,
        level: int = logging.ERROR,
        channel: Optional[str] = None,
        username: Optional[str] = None,
        icon_emoji: Optional[str] = None,
        timeout: int = 5,
        dry_run: bool = False,
        max_message_length: int = 3000,
        use_blocks: bool = True,
    ):
        super().__init__(level)
        if not webhook_url and not dry_run:
            raise ValueError("webhook_url is required unless dry_run=True.")
        self.webhook_url    = webhook_url
        self.channel        = channel
        self.username       = username
        self.icon_emoji     = icon_emoji
        self.timeout        = timeout
        self.dry_run        = dry_run
        self.use_blocks     = use_blocks
        self.max_message_length = max_message_length
        
        # Emoji by level for quick visual scanning.
        self.level_emoji = {
            logging.CRITICAL: "🚨",
            logging.ERROR: "❌",
            logging.WARNING: "⚠️",
            logging.INFO: "ℹ️",
            logging.DEBUG: "🐞",
        }

    def _build_text(self, record: logging.LogRecord) -> str:
        # Use handler's formatter if set; otherwise default.
        if self.formatter:
            formatted = self.format(record)
        else:
            default_fmt = logging.Formatter("%(asctime)s [%(name)s] %(levelname)s: %(message)s")
            formatted = default_fmt.format(record)
        # Truncate to keep payload reasonable.
        if len(formatted) > self.max_message_length:
            formatted = formatted[: self.max_message_length - 1] + "…"
        # Prepend emoji for level.
        emoji = self.level_emoji.get(record.levelno, "")
        return f"{emoji} {formatted}" if emoji else formatted

    def _payload(self, text: str) -> Dict[str, Any]:
        if self.use_blocks:
            blocks = [
                {
                    "type": "section",
                    "text": {"type": "mrkdwn", "text": f"```{text}```"},
                }
            ]
            payload: Dict[str, Any] = {"blocks": blocks}
        else:
            payload = {"text": text}

        # Optional fields. Note that some webhook apps ignore channel overrides.
        if self.channel:
            payload["channel"] = self.channel
        if self.username:
            payload["username"] = self.username
        if self.icon_emoji:
            payload["icon_emoji"] = self.icon_emoji
        return payload

    def emit(self, record: logging.LogRecord) -> None:
        try:
            text = self._build_text(record)
            payload = self._payload(text)
            body = json.dumps(payload).encode("utf-8")

            if self.dry_run:
                # Print payload instead of sending to Slack.
                print("[SlackWebhookHandler dry_run] Payload:", json.dumps(payload, ensure_ascii=False))
                return

            req = Request(self.webhook_url, data=body, headers={"Content-Type": "application/json"})
            with urlopen(req, timeout=self.timeout) as resp:
                # Consume response to avoid ResourceWarnings.
                resp.read()
        except (HTTPError, URLError, TimeoutError, Exception):
            # Never raise from a logging handler; delegate to logging's error handler.
            self.handleError(record)


# -----------------------------
# Example setup and quick test.
# -----------------------------
# 1) Provide your webhook via env var or direct string.
WEBHOOK_URL = os.getenv("SLACK_WEBHOOK_URL", "")  # Example: "https://hooks.slack.com/services/T000/B000/XXXXXXXX"

# 2) Clean root config to avoid Jupyter duplicates.
logging.basicConfig(handlers=[], force=True)

# 3) Build console handler for local visibility.
console = logging.StreamHandler()
console.setFormatter(logging.Formatter("%(asctime)s [%(name)s] %(levelname)s: %(message)s", "%H:%M:%S"))

# 4) Build Slack handler.
slack = SlackWebhookHandler(
    webhook_url=WEBHOOK_URL,
    level=logging.WARNING,            # Send WARNING and above to Slack.
    channel=None,                     # Optional. Webhook may ignore overrides.
    username="MyApp Logger",          # Optional display name.
    icon_emoji=":robot_face:",        # Optional icon.
    timeout=5,
    dry_run=True,                     # Set to False to actually send to Slack.
    max_message_length=3000,
    use_blocks=True,
)
slack.setFormatter(logging.Formatter("%(asctime)s [%(name)s] %(levelname)s: %(message)s", "%Y-%m-%d %H:%M:%S"))
slack.addFilter(DedupFilter(window_seconds=30, key="name+message"))  # Prevent repeat storms.

# 5) Attach to your app logger.
logger = logging.getLogger("tms.backend")
logger.setLevel(logging.DEBUG)
logger.addHandler(console)
logger.addHandler(slack)
logger.propagate = False  # Keep from bubbling to root if you attach handlers there too.

# 6) Demo messages.
logger.debug("Debug detail that stays local.")
logger.info("Startup complete.")
logger.warning("Upstream latency is high.")
logger.error("Failed to fetch route ETA for order_id=12345.")
logger.critical("Database unavailable. Failing open for read-only mode.")
